In [1]:
!pip install tensorflow

  Using cached tensorflow-2.7.0-cp38-cp38-win_amd64.whl (430.8 MB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached flatbuffers-2.0-py2.py3-none-any.whl (26 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached grpcio-1.43.0-cp38-cp38-win_amd64.whl (3.4 MB)
  Using cached tensorflow_estimator-2.7.0-py2.py3-none-any.whl (463 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached tensorboard-2.7.0-py3-none-any.whl (5.8 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached tensorflow_io_gcs_filesystem-0.23.1-cp38-cp38-win_amd64.whl (1.5 MB)
  Using cached keras-2.7.0-py2.py3-none-any.whl (1.3 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached google_auth-2.3.3-py2.py3-none-any.whl (155 kB)
  Using cached Markdown-3.3.6-py3-none-any.whl (97 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  U

In [1]:
import mediapipe as mp
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib as plt
import cv2

In [3]:
import os
rootdir = 'D:\ML\ASLDetection\Data'

all_dir = os.listdir(rootdir)
X = []
Y = []
for name in all_dir:
    for filename in os.listdir(os.path.join(rootdir, name)):
        f = os.path.join(os.path.join(rootdir, name), filename)
        if os.path.isfile(f):
            mp_hands = mp.solutions.hands
            with mp_hands.Hands(static_image_mode=True) as hands:
                img = cv2.imread(f)
                img = cv2.flip(img,1)
                img_height,img_width,_ = img.shape
                annotated_image = img.copy()
                results = hands.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                keypoints = []
                if not results.multi_hand_world_landmarks:
                    print(f)
                    continue
                for hand_world_landmarks in results.multi_hand_world_landmarks:
                    for data_point in hand_world_landmarks.landmark:
                        keypoints.append(data_point.x)
                        keypoints.append(data_point.y)
                        keypoints.append(data_point.z)
                X.append(keypoints)
                Y.append(name)
X_train = pd.DataFrame(X)

D:\ML\ASLDetection\Data\D\D0004_test.jpg
D:\ML\ASLDetection\Data\D\D0005_test.jpg
D:\ML\ASLDetection\Data\I\I0003_test.jpg
D:\ML\ASLDetection\Data\I\I0004_test.jpg
D:\ML\ASLDetection\Data\I\I0007_test.jpg
D:\ML\ASLDetection\Data\I\I0008_test.jpg
D:\ML\ASLDetection\Data\I\I0009_test.jpg
D:\ML\ASLDetection\Data\I\I0010_test.jpg
D:\ML\ASLDetection\Data\I\I0011_test.jpg
D:\ML\ASLDetection\Data\I\I0012_test.jpg
D:\ML\ASLDetection\Data\I\I0013_test.jpg
D:\ML\ASLDetection\Data\I\I0025_test.jpg
D:\ML\ASLDetection\Data\K\K0001_test.jpg
D:\ML\ASLDetection\Data\K\K0002_test.jpg
D:\ML\ASLDetection\Data\K\K0030_test.jpg
D:\ML\ASLDetection\Data\M\M0001_test.jpg
D:\ML\ASLDetection\Data\M\M0010_test.jpg
D:\ML\ASLDetection\Data\M\M0030_test.jpg
D:\ML\ASLDetection\Data\N\N0022_test.jpg
D:\ML\ASLDetection\Data\N\N0023_test.jpg
D:\ML\ASLDetection\Data\O\O0001_test.jpg
D:\ML\ASLDetection\Data\O\O0003_test.jpg
D:\ML\ASLDetection\Data\O\O0021_test.jpg
D:\ML\ASLDetection\Data\O\O0022_test.jpg
D:\ML\ASLDetecti

In [4]:
X_train.shape

(747, 63)

In [25]:
encoding = {
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6,
    'G': 7,
    'H': 8,
    'I': 9,
    'J': 10,
    'K': 11,
    'L': 12,
    'M': 13,
    'N': 14,
    'O': 15,
    'P': 16,
    'Q': 17,
    'R': 18,
    'S': 19,
    'T': 20,
    'U': 21,
    'V': 22,
    'W': 23,
    'X': 24,
    'Y': 25,
    'Z': 26,    
}

new_Y = Y.copy()
for idx,letter in enumerate(Y):
    new_Y[idx] = encoding[letter] - 1
    
Y_train = pd.DataFrame(new_Y)

In [26]:

from keras.utils import np_utils
dummy_y = np_utils.to_categorical(Y_train)

dummy_y.shape

(747, 26)

In [63]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(units=128, activation='relu', input_shape=[X_train.shape[1]]),
    layers.BatchNormalization(),
    layers.Dense(units=128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(units = 64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(units = 48, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(units=26, activation='sigmoid')
])

model.compile(optimizer = 'adam', loss='categorical_crossentropy')

In [64]:
from tensorflow.keras import callbacks
callback = callbacks.EarlyStopping(patience = 5,
                                  min_delta = 0.001,
                                  restore_best_weights = True)

In [65]:
history = model.fit(X_train,dummy_y,epochs = 100)

Epoch 1/100
24/24 [==============================] - 1s 2ms/step - loss: 1.9731
Epoch 2/100
24/24 [==============================] - 0s 2ms/step - loss: 0.7734
Epoch 3/100
24/24 [==============================] - 0s 2ms/step - loss: 0.4676
Epoch 4/100
24/24 [==============================] - 0s 2ms/step - loss: 0.3451
Epoch 5/100
24/24 [==============================] - 0s 2ms/step - loss: 0.2548
Epoch 6/100
24/24 [==============================] - 0s 2ms/step - loss: 0.2095
Epoch 7/100
24/24 [==============================] - 0s 2ms/step - loss: 0.1793
Epoch 8/100
24/24 [==============================] - 0s 2ms/step - loss: 0.1567
Epoch 9/100
24/24 [==============================] - 0s 2ms/step - loss: 0.1395
Epoch 10/100
24/24 [==============================] - 0s 2ms/step - loss: 0.1368
Epoch 11/100
24/24 [==============================] - 0s 2ms/step - loss: 0.1346
Epoch 12/100
24/24 [==============================] - 0s 2ms/step - loss: 0.1217
Epoch 13/100
24/24 [=================

In [69]:
x = [X_train.iloc[100].array.astype('float64').reshape(-1,63)]
y = model.predict(x)

y

array([[0.5046299 , 0.5096518 , 0.9219891 , 0.99999106, 0.1116749 ,
        0.6392288 , 0.685708  , 0.598137  , 0.26262927, 0.60612494,
        0.04539073, 0.12906161, 0.49065563, 0.75605464, 0.9291538 ,
        0.60856164, 0.5228858 , 0.2628013 , 0.01741427, 0.10703269,
        0.694689  , 0.15397531, 0.7882817 , 0.0961715 , 0.6493746 ,
        0.5452549 ]], dtype=float32)

In [70]:
y = [i > 0.999 for i in y]

y

[array([False, False, False,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False])]

In [71]:
model.save("ASL_model.hdf5")